<a href="https://colab.research.google.com/github/ricardosavaris/alura/blob/main/Foxo1ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Section 1: Setup and Data Loading ---

# Import necessary libraries for data analysis, visualization, and statistical modeling.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pingouin as pg
from scipy.stats import normaltest, levene, kruskal, shapiro
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from google.colab import drive # Keep if accessing from Drive
from google.colab import auth # Keep if accessing from Drive
import gspread # Keep if accessing from Drive
from scipy.stats import f_oneway, kruskal # Import f_oneway
import statsmodels.stats.oneway as oneway

# Define the path to the uploaded Excel file
file_path = 'FOXO1_raw_data.xlsx' # Assuming the file is in the default /content/ directory

# Read the Excel file into a pandas DataFrame
try:
    df = pd.read_excel(file_path, engine='openpyxl')
    print("DataFrame loaded successfully from local file:")
    display(df.head())
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

**Reasoning**:
Arrange the remaining code cells in a logical order that reflects the steps of the analysis presented in the manuscript (e.g., Data Loading -> Data Preparation -> Descriptive Statistics -> Group Comparisons -> Regression Analysis 1 -> Regression Analysis 2 -> Table Generation).

In [ ]:
# Define the variables to analyze
variables_to_analyze = ['Age', 'BMI', 'End Tickness', 'Serum E2 (pg/ml)', 'Serum P4 (ng/ml)', 'PGR QPath', 'FOXO1 Qpath']

# Define the groups
groups = df['Group'].unique()

# Dictionary to store the results
summary_data = {}

for group in groups:
    group_df = df[df['Group'] == group].copy()
    summary_data[group] = {}

    for variable in variables_to_analyze:
        if variable in group_df.columns:
            data = group_df[variable].dropna()

            if not data.empty:
                # Calculate descriptive statistics
                mean = data.mean()
                std_dev = data.std()
                median = data.median()
                data_range = data.max() - data.min()
                n = len(data)

                # Perform normality test
                if n >= 8: # D'Agostino's test requires at least 8 data points
                    k2, p_value = normaltest(data)
                    normality_passed = 'Yes' if p_value > 0.05 else 'No' # Assuming alpha = 0.05
                else:
                    normality_passed = 'Not enough data (n < 8)'

                summary_data[group][variable] = {
                    'n': n,
                    'Mean': mean,
                    'Std Dev': std_dev,
                    'Median': median,
                    'Range': data_range,
                    'Normal Distribution (p>0.05)': normality_passed
                }
            else:
                summary_data[group][variable] = 'No data available'
        else:
            summary_data[group][variable] = 'Variable not found'


# Prepare data for a consolidated table
all_summary_data = []
for group, variables_data in summary_data.items():
    for variable, stats in variables_data.items():
        if isinstance(stats, dict):
            row = {'Group': group, 'Variable': variable, **stats}
            all_summary_data.append(row)
        else:
             all_summary_data.append({'Group': group, 'Variable': variable, 'Result': stats})


# Create a pandas DataFrame for the summary table
summary_table = pd.DataFrame(all_summary_data)

# Display the summary table
print("Descriptive Statistics and Normality Test Summary Table:")
display(summary_table)

## Check for homoscedasticity

### Subtask:
For variables that are approximately normally distributed in all three groups, perform a test for equal variances (e.g., Levene's test) across the groups.


**Reasoning**:
Perform Levene's test for equal variances for 'Age' and 'Serum E2 (pg/ml)' across the three groups.



In [ ]:
from scipy.stats import levene

# Identify variables that were approximately normally distributed in all groups
normally_distributed_vars = ['Age', 'Serum E2 (pg/ml)']

# Define the groups
groups = df['Group'].unique()

# Perform Levene's test for each normally distributed variable
for variable in normally_distributed_vars:
    group_data = [df[df['Group'] == group][variable].dropna() for group in groups]

    # Ensure there are at least two groups with data for the variable
    if len(group_data) >= 2:
        statistic, p_value = levene(*group_data)
        print(f"Levene's test for {variable}:")
        print(f"  Statistic: {statistic:.4f}")
        print(f"  P-value: {p_value:.4f}")
        print("-" * 30)
    else:
        print(f"Not enough groups with data for {variable} to perform Levene's test.")
        print("-" * 30)

## Select and perform appropriate test

### Subtask:
Perform the appropriate statistical test (ANOVA, Welch's ANOVA, or Kruskal-Wallis) for each variable based on normality and homoscedasticity results, comparing the three groups.


**Reasoning**:
Based on the normality and homoscedasticity results, perform the appropriate statistical tests for each variable. For 'Age', perform one-way ANOVA. For 'Serum E2 (pg/ml)', perform Welch's ANOVA. For the remaining variables, perform Kruskal-Wallis.



In [ ]:
# Define the variables and the appropriate tests
variables_to_test = {
    'Age': 'ANOVA',
    'BMI': 'Kruskal-Wallis',
    'End Tickness': 'Kruskal-Wallis',
    'Serum E2 (pg/ml)': 'Welch-ANOVA',
    'Serum P4 (ng/ml)': 'Kruskal-Wallis',
    'PGR QPath': 'Kruskal-Wallis',
    'FOXO1 Qpath': 'Kruskal-Wallis'
}

# Define the groups
groups = df['Group'].unique()

# Perform the appropriate test for each variable
for variable, test_type in variables_to_test.items():
    print(f"Performing {test_type} test for {variable}:")

    # Prepare data for the test, dropping NaN values for the current variable
    group_data = [df[df['Group'] == group][variable].dropna() for group in groups]

    # Remove empty groups if any
    group_data = [data for data in group_data if not data.empty]

    if len(group_data) < 2:
        print(f"  Not enough groups with data for {variable} to perform the test.")
    else:
        if test_type == 'ANOVA':
            if len(group_data) >= 2: # Ensure at least two groups with data
                statistic, p_value = f_oneway(*group_data)
                print(f"  ANOVA Statistic: {statistic:.4f}")
                print(f"  ANOVA P-value: {p_value:.4f}")
            else:
                print(f"  Not enough groups with data for {variable} to perform ANOVA.")
        elif test_type == 'Welch-ANOVA':
             if len(group_data) >= 2: # Ensure at least two groups with data
                # Use pingouin.welch_anova
                # Create a temporary DataFrame for pingouin
                temp_df = pd.DataFrame({variable: pd.concat(group_data), 'Group': df['Group'].loc[pd.concat(group_data).index]})
                anova_results = pg.welch_anova(data=temp_df, dv=variable, between='Group')
                f_statistic = anova_results['F'].iloc[0]
                p_value = anova_results['p-unc'].iloc[0]
                print(f"  Welch's ANOVA Statistic: {f_statistic:.4f}")
                print(f"  Welch's ANOVA P-value: {p_value:.4f}")
             else:
                print(f"  Not enough groups with data for {variable} to perform Welch's ANOVA.")
        elif test_type == 'Kruskal-Wallis':
            if len(group_data) >= 2: # Ensure at least two groups with data
                statistic, p_value = kruskal(*group_data)
                print(f"  Kruskal-Wallis Statistic: {statistic:.4f}")
                print(f"  Kruskal-Wallis P-value: {p_value:.4f}")
            else:
                 print(f"  Not enough groups with data for {variable} to perform Kruskal-Wallis test.")
    print("-" * 30)

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact

# Create a contingency table of 'Group' and 'HE Dating'
contingency_table = pd.crosstab(df['Group'], df['HE Dating'])

print("Contingency Table (Group vs. HE Dating):")
display(contingency_table)

# Perform Chi-squared test
# Chi-squared test is generally appropriate for larger sample sizes.
# If any cell has an expected frequency less than 5, Fisher's exact test is more suitable.
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("\nChi-squared Test Results:")
print(f"  Chi-squared Statistic: {chi2:.4f}")
print(f"  P-value: {p:.4f}")
print(f"  Degrees of Freedom: {dof}")
# print("  Expected Frequencies Table:")
# display(pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns))

# Check if Fisher's exact test is more appropriate (e.g., for small sample sizes)
# A common rule of thumb is if any expected cell count is less than 5.
# We can check this from the 'expected' array from chi2_contingency.
if (expected < 5).any():
    print("\nWarning: Some expected cell counts are less than 5. Fisher's exact test may be more appropriate.")
    # For a 3x2 table, Fisher's exact test can be computationally intensive or
    # might require a different implementation than scipy.stats.fisher_exact which is for 2x2.
    # However, let's perform it if possible or note the need for specialized test for larger tables.
    # SciPy's fisher_exact is strictly for 2x2 tables. For larger tables, other libraries or methods are needed.
    # We'll stick with the chi-squared output and the warning for now, as it's a common approach for larger tables with some small counts.
    pass # We will rely on the chi-squared test results and the warning.
else:
    print("\nFisher's exact test is not required based on expected cell counts.")

Creating a Table 1. Characteristics of the studied population

In [ ]:
# Define the variables and their desired format (mean-SD or median-range)
variable_formats = {
    'Age': 'mean-SD',
    'BMI': 'median-range',
    'End Tickness': 'median-range',
    'Serum E2 (pg/ml)': 'mean-SD',
    'Serum P4 (ng/ml)': 'median-range',
    'PGR QPath': 'mean-SD',
    'FOXO1 Qpath': 'median-range'
}

# Define the variables and the appropriate tests (from previous analysis in cell 2c387b67)
variables_to_test = {
    'Age': 'ANOVA',
    'BMI': 'Kruskal-Wallis',
    'End Tickness': 'Kruskal-Wallis',
    'Serum E2 (pg/ml)': 'Welch-ANOVA',
    'Serum P4 (ng/ml)': 'Kruskal-Wallis',
    'PGR QPath': 'Kruskal-Wallis',
    'FOXO1 Qpath': 'Kruskal-Wallis'
}

# Dictionary to store the formatted data for the table
table_data = {}

# Populate the table data with variable names
for variable in variable_formats.keys():
    table_data[variable] = {}
    table_data[variable]['Variables/Characteristics'] = variable

# Extract data from the summary_table and format it
for index, row in summary_table.iterrows():
    group = row['Group']
    variable = row['Variable']
    mean = row['Mean']
    std_dev = row['Std Dev']
    median = row['Median']
    data_range = row['Range'] # Note: summary_table has Range, not IQR

    if variable in variable_formats:
        if variable_formats[variable] == 'mean-SD':
            formatted_value = f"{mean:.2f} ± {std_dev:.2f}"
        elif variable_formats[variable] == 'median-range':
            formatted_value = f"{median:.2f} ({data_range:.2f})" # Using Range as per summary_table

        if group not in table_data[variable]:
             table_data[variable][group] = formatted_value
        else:
             # Handle cases where group name might have extra spaces
             table_data[variable][group.strip()] = formatted_value


# Add P-value and test used from the results in cell 2c387b67
# We need to access the p-values calculated in cell 2c387b67.
# Assuming the p-values and test types from cell 2c387b67 are available in memory
# We will manually add them based on the output of cell 2c387b67

# Manually adding p-values and test names based on the output of cell 2c387b67
# Note: This is a temporary solution. In a real scenario, we would store these results
# in a more accessible structure in cell 2c387b67.
p_values = {
    'Age': 0.0870,
    'BMI': 0.2770,
    'End Tickness': 0.1638,
    'Serum E2 (pg/ml)': 0.0000, # Approximate p-value from Welch's ANOVA output
    'Serum P4 (ng/ml)': 0.0001,
    'PGR QPath': 0.0211,
    'FOXO1 Qpath': 0.0001
}

for variable in variable_formats.keys():
    if variable in p_values:
        table_data[variable]['P-value'] = f"{p_values[variable]:.4f}"
        table_data[variable]['test used'] = variables_to_test[variable]


# Convert the dictionary to a list of dictionaries and then to a DataFrame
table_rows = [data for data in table_data.values()]
table1_df = pd.DataFrame(table_rows)

# Reorder columns to match the requested format
column_order = ['Variables/Characteristics', 'Control', 'IVF  ', 'HRT  ', 'P-value', 'test used']
table1_df = table1_df[column_order]

# Display the Table 1
print("Table 1:")
display(table1_df)

In [ ]:
# Define the variables to test for normality
variables_for_normality_check = ['FOXO1 Qpath', 'PGR QPath', 'Serum E2 (pg/ml)', 'Serum P4 (ng/ml)']

print("Normality Test Results (D'Agostino's K-squared test) for selected variables:")
print("-" * 70)

for variable in variables_for_normality_check:
    if variable in df.columns:
        # Drop NaN values before testing
        data = df[variable].dropna()

        if len(data) > 8: # D'Agostino's test requires at least 8 data points
            k2, p_value = normaltest(data)
            print(f"{variable}:")
            print(f"  K2 Statistic: {k2:.4f}")
            print(f"  P-value: {p_value:.4f}")
            if p_value < 0.05:
                print("  Result: Not normally distributed (p < 0.05)")
            else:
                print("  Result: Normally distributed (p >= 0.05)")
        else:
            print(f"{variable}: Not enough data (n < 8) to perform D'Agostino's test.")
    else:
        print(f"Variable not found: {variable}")
    print("-" * 70)

Using different algorithms to achieve normal distribution for the variables : 'FOXO1 Qpath', 'PGR QPath', 'Serum E2 (pg/ml)', 'Serum P4 (ng/ml)'

In [ ]:
import numpy as np
from scipy.stats import normaltest, boxcox, rankdata, norm
import pandas as pd

# Define the variables to transform and test
variables_to_transform = ['FOXO1 Qpath', 'PGR QPath', 'Serum E2 (pg/ml)', 'Serum P4 (ng/ml)']

# Create a copy of the DataFrame to add transformed columns
df_transformed_vars = df.copy()

print("Exploring transformations for normality:")
print("-" * 70)

for variable in variables_to_transform:
    original_col = variable

    if original_col in df_transformed_vars.columns:
        data = df_transformed_vars[original_col].dropna()

        if not data.empty and len(data) > 1:
            print(f"Applying transformations to '{original_col}':")

            # --- Log Transformation ---
            transformed_col_log = f'{variable}_log'
            if (data <= 0).any():
                min_positive = data[data > 0].min()
                constant = min_positive / 2 if pd.notna(min_positive) else 1e-6
                df_transformed_vars[transformed_col_log] = np.log(data + constant)
                print(f"  Log transformation applied with constant {constant:.6f}.")
            else:
                df_transformed_vars[transformed_col_log] = np.log(data)
                print(f"  Log transformation applied.")

            # Test Log-transformed variable for normality
            data_transformed_log = df_transformed_vars[transformed_col_log].dropna()
            if len(data_transformed_log) > 8:
                k2_log, p_value_log = normaltest(data_transformed_log)
                normality_log = 'Normally distributed (p >= 0.05)' if p_value_log >= 0.05 else 'Not normally distributed (p < 0.05)'
                print(f"  Normality Test (Log): K2={k2_log:.4f}, P={p_value_log:.4f} - {normality_log}")
            else:
                 print(f"  Not enough data (n < 8) for Log Normality Test.")


            # --- Box-Cox Transformation ---
            if (data > 0).all():
                transformed_col_boxcox = f'{variable}_boxcox'
                transformed_data_boxcox, lambda_boxcox = boxcox(data)
                df_transformed_vars[transformed_col_boxcox] = transformed_data_boxcox
                print(f"  Box-Cox transformation applied (lambda={lambda_boxcox:.4f}).")

                # Test Box-Cox transformed variable for normality
                data_transformed_boxcox = df_transformed_vars[transformed_col_boxcox].dropna()
                if len(data_transformed_boxcox) > 8:
                    k2_boxcox, p_value_boxcox = normaltest(data_transformed_boxcox)
                    normality_boxcox = 'Normally distributed (p >= 0.05)' if p_value_boxcox >= 0.05 else 'Not normally distributed (p < 0.05)'
                    print(f"  Normality Test (Box-Cox): K2={k2_boxcox:.4f}, P={p_value_boxcox:.4f} - {normality_boxcox}")
                else:
                    print(f"  Not enough data (n < 8) for Box-Cox Normality Test.")
            else:
                 print(f"  Box-Cox transformation skipped for '{original_col}' due to non-positive values.")

            # --- Normal Quantile Transformation (NQT) ---
            # NQT can be applied to data with zeros or negatives.
            if len(data) >= 2: # NQT requires at least 2 data points
                transformed_col_nqt = f'{variable}_nqt'
                # Calculate ranks
                ranks = rankdata(data)
                # Calculate quantiles
                quantiles = (ranks - 0.5) / len(ranks)
                # Apply inverse of normal CDF
                transformed_data_nqt = norm.ppf(quantiles)
                df_transformed_vars[transformed_col_nqt] = transformed_data_nqt
                print(f"  Normal Quantile Transformation (NQT) applied.")

                # Test NQT transformed variable for normality
                data_transformed_nqt = df_transformed_vars[transformed_col_nqt].dropna()
                if len(data_transformed_nqt) > 8:
                    k2_nqt, p_value_nqt = normaltest(data_transformed_nqt)
                    normality_nqt = 'Normally distributed (p >= 0.05)' if p_value_nqt >= 0.05 else 'Not normally distributed (p < 0.05)'
                    print(f"  Normality Test (NQT): K2={k2_nqt:.4f}, P={p_value_nqt:.4f} - {normality_nqt}")
                else:
                    print(f"  Not enough data (n < 8) for NQT Normality Test.")
            else:
                 print(f"  Not enough data ({len(data)} < 2) in '{original_col}' to perform NQT.")


        else:
            print(f"Not enough data ({len(data)} < 2) in '{original_col}' to perform transformations.")
    else:
        print(f"Variable not found: {original_col}")
    print("-" * 70)

# Display the head of the DataFrame with new transformed columns
print("\nDataFrame head with transformed variables:")
display(df_transformed_vars.head())

In [ ]:
#df_regression_nqt is available from the previous step (Prepare Data - NQT).

if 'df_regression_nqt' not in locals():
    print("df_regression_nqt not found, creating it from df_transformed_vars...")
    df_regression_nqt = df_transformed_vars[['FOXO1 Qpath_nqt', 'PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']].dropna()


# Define independent and dependent variables using the NQT transformed data
independent_vars_nqt = ['PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']
dependent_var_nqt = 'FOXO1 Qpath_nqt'

# Create scatter plots
print("Scatter Plots to Check Linearity (NQT Transformed Variables):")
for var in independent_vars_nqt:
    plt.figure(figsize=(8, 6))
    plt.scatter(df_regression_nqt[var], df_regression_nqt[dependent_var_nqt])
    plt.xlabel(var)
    plt.ylabel(dependent_var_nqt)
    plt.title(f'Scatter Plot of {dependent_var_nqt} vs. {var}')
    plt.grid(True)
    plt.show()

print("\nVisual inspection of scatter plots is needed to assess linearity.")

In [ ]:
# Assuming df_regression_nqt is available from the previous step (Prepare Data - NQT).

if 'df_regression_nqt' not in locals():
    print("df_regression_nqt not found, creating it from df_transformed_vars...")
    df_regression_nqt = df_transformed_vars[['FOXO1 Qpath_nqt', 'PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']].dropna()


# Define the independent variables (X) and the dependent variable (y) using the NQT transformed data
X_nqt = df_regression_nqt[['PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']]
y_nqt = df_regression_nqt['FOXO1 Qpath_nqt']

# Add a constant to the independent variables
X_nqt = sm.add_constant(X_nqt)

# Fit the linear regression model with NQT transformed variables
# We fit the model here to get residuals and predicted values for homoscedasticity check.
model_nqt = sm.OLS(y_nqt, X_nqt).fit()

# Calculate the predicted values and residuals for the NQT model
predicted_values_nqt = model_nqt.predict(X_nqt)
residuals_nqt = model_nqt.resid

# Create a scatter plot of residuals vs. predicted values for the NQT model
print("Residuals vs. Predicted Values Plot (NQT Transformed):")
plt.figure(figsize=(10, 6))
plt.scatter(predicted_values_nqt, residuals_nqt)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values (NQT Transformed)')
plt.ylabel('Residuals (NQT Transformed)')
plt.title('Residuals vs. Predicted Values Plot (NQT Transformed)')
plt.grid(True)
plt.show()

# Perform the Breusch-Pagan test on the NQT model residuals
bp_test_nqt = het_breuschpagan(residuals_nqt, X_nqt)

# Print the results of the Breusch-Pagan test for NQT model residuals
print("\nBreusch-Pagan Test Results (NQT Model):")
print(f"  Lagrange multiplier statistic: {bp_test_nqt[0]:.4f}")
print(f"  P-value: {bp_test_nqt[1]:.4f}")
print(f"  F-statistic: {bp_test_nqt[2]:.4f}")
print(f"  F-statistic P-value: {bp_test_nqt[3]:.4f}")

print("\nVisual inspection of the plot and the Breusch-Pagan test results are needed to assess homoscedasticity.")

Visual inspection reveals that the data passed the Homoscedasticity test

In [ ]:
# Assuming residuals_nqt is available from the previous step (Homoscedasticity check).
if 'residuals_nqt' not in locals():
    print("residuals_nqt not found, re-running regression to get residuals...")
    if 'df_regression_nqt' not in locals():
        print("df_regression_nqt not found, creating it from df_transformed_vars...")
        # Assuming df_transformed_vars is available from cell SXDUqesKDYE5:
        df_regression_nqt = df_transformed_vars[['FOXO1 Qpath_nqt', 'PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']].dropna()

    X_nqt = df_regression_nqt[['PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']]
    y_nqt = df_regression_nqt['FOXO1 Qpath_nqt']
    X_nqt = sm.add_constant(X_nqt)
    model_nqt = sm.OLS(y_nqt, X_nqt).fit()
    residuals_nqt = model_nqt.resid


print("Checking Normality of Residuals (NQT Transformed Model):")

# Generate a histogram of the residuals for the NQT model
plt.figure(figsize=(10, 6))
plt.hist(residuals_nqt, bins=20, density=True, alpha=0.7)
plt.xlabel('Residuals (NQT Transformed)')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals (NQT Transformed Model)')
plt.grid(True)
plt.show()

# Create a Q-Q plot of the residuals for the NQT model
sm.qqplot(residuals_nqt, line='s')
plt.title('Q-Q Plot of Residuals (NQT Transformed Model)')
plt.show()

# Perform the Shapiro-Wilk test on the NQT model residuals
shapiro_test_stat_nqt, shapiro_p_value_nqt = shapiro(residuals_nqt)

# Print the results of the Shapiro-Wilk test for NQT model residuals
print("\nShapiro-Wilk Test Results for Residuals (NQT Model):")
print(f"  Test Statistic: {shapiro_test_stat_nqt:.4f}")
print(f"  P-value: {shapiro_p_value_nqt:.4f}")

print("\nVisual inspection of the plots and the Shapiro-Wilk test results are needed to assess normality of residuals.")

Passed the residual normality test

In [ ]:
#Verifying multicolinearity Model 1:
if 'df_regression_nqt' not in locals():
    print("df_regression_nqt not found, creating it from df_transformed_vars...")
    df_regression_nqt = df_transformed_vars[['FOXO1 Qpath_nqt', 'PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']].dropna()


# Create a DataFrame with the NQT transformed independent variables (X_nqt)
X_nqt = df_regression_nqt[['PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt','Serum P4 (ng/ml)_nqt']]

# Add a constant term to the independent variables
X_nqt = sm.add_constant(X_nqt)

# Calculate the VIF for each independent variable
vif_data_nqt = pd.DataFrame()
vif_data_nqt["variable"] = X_nqt.columns
vif_data_nqt["VIF"] = [variance_inflation_factor(X_nqt.values, i)
                          for i in range(X_nqt.shape[1])]

# Print the VIF values
print("Variance Inflation Factor (VIF) for NQT transformed independent variables:")
display(vif_data_nqt)

print("\nVIF values are generally considered problematic if they are above 5 or 10. Low VIF values indicate that multicollinearity is not a significant issue.")

In [ ]:
##Verifying multicolinearity Model 2
if 'df_regression_nqt' not in locals():
    print("df_regression_nqt not found, creating it from df_transformed_vars...")
    df_regression_nqt = df_transformed_vars[['FOXO1 Qpath_nqt', 'PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']].dropna()


# Create a DataFrame with the NQT transformed independent variables (X_nqt) for the second model
X_nqt_model2 = df_regression_nqt[['Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt', 'FOXO1 Qpath_nqt']]

# Add a constant term to the independent variables
X_nqt_model2 = sm.add_constant(X_nqt_model2)

# Calculate the VIF for each independent variable
vif_data_nqt_model2 = pd.DataFrame()
vif_data_nqt_model2["variable"] = X_nqt_model2.columns
vif_data_nqt_model2["VIF"] = [variance_inflation_factor(X_nqt_model2.values, i)
                              for i in range(X_nqt_model2.shape[1])]

# Print the VIF values
print("Variance Inflation Factor (VIF) for NQT transformed independent variables (PGR QPath_nqt as Dependent):")
display(vif_data_nqt_model2)

print("\nVIF values are generally considered problematic if they are above 5 or 10. Low VIF values indicate that multicollinearity is not a significant issue.")

In [ ]:
#MODEL 1 FOXO1 as dependent variable
if 'df_regression_nqt' not in locals():
    print("df_regression_nqt not found, creating it from df_transformed_vars...")
    df_regression_nqt = df_transformed_vars[['FOXO1 Qpath_nqt', 'PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']].dropna()


# Define the independent variables (X) and the dependent variable (y) using the NQT transformed data
X_nqt = df_regression_nqt[['PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']]
y_nqt = df_regression_nqt['FOXO1 Qpath_nqt']

# Add a constant to the independent variables
X_nqt = sm.add_constant(X_nqt)

# Fit the multiple linear regression model
model_nqt = sm.OLS(y_nqt, X_nqt).fit()

# Print the summary of the regression results
print("Multiple Linear Regression Results (Model 1 - NQT Transformed Variables):")
print(model_nqt.summary())

print("\nInterpretation of the results, including coefficients, p-values, R-squared, and assumption checks (linearity, homoscedasticity, normality of residuals, multicollinearity), is needed.")

In [ ]:
#Model 2: PGR as dependent variable:
if 'df_regression_nqt' not in locals():
    print("df_regression_nqt not found, creating it from df_transformed_vars...")
    df_regression_nqt = df_transformed_vars[['FOXO1 Qpath_nqt', 'PGR QPath_nqt', 'Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt']].dropna()


# Define the independent variables (X) and the dependent variable (y) using the NQT transformed data
X_nqt = df_regression_nqt[['Serum E2 (pg/ml)_nqt', 'Serum P4 (ng/ml)_nqt', 'FOXO1 Qpath_nqt']]
y_nqt = df_regression_nqt['PGR QPath_nqt'] # Dependent variable is PGR QPath_nqt

# Add a constant to the independent variables
X_nqt = sm.add_constant(X_nqt)

# Fit the multiple linear regression model
model_pgr_nqt = sm.OLS(y_nqt, X_nqt).fit() # Naming the model specifically for PGR as dependent

# Print the summary of the regression results
print("Multiple Linear Regression Results (Model 2 - PGR QPath as Dependent Variable - NQT Transformed Variables):")
print(model_pgr_nqt.summary())

print("\nInterpretation of the results, including coefficients, p-values, R-squared, and assumption checks (linearity, homoscedasticity, normality of residuals, multicollinearity), is needed.")

## Explanation of Multiple Linear Regression Results (NQT Transformed Variables)

Here is an explanation of the results from the two multiple linear regression models performed using the Normal Quantile Transformed (NQT) variables:

### Model 1: Dependent Variable = FOXO1 Qpath\_nqt

(Based on the output of cell `VeMDPpfoNY7V`)

*   **Overall Model Fit:**
    *   **R-squared = 0.292:** This means that approximately 29.2% of the variance in the Normal Quantile Transformed FOXO1 Qpath can be explained by the NQT transformed independent variables (PGR QPath\_nqt, Serum E2 (pg/ml)\_nqt, and Serum P4 (ng/ml)\_nqt).
    *   **Adjusted R-squared = 0.238:** This is a slightly adjusted R-squared value that accounts for the number of predictors in the model.
    *   **F-statistic = 5.374, Prob (F-statistic) = 0.00341:** The overall model is statistically significant (p < 0.05). This indicates that at least one of the independent variables significantly predicts the NQT transformed FOXO1 Qpath.

*   **Individual Predictors:**
    *   **const (Intercept):** The intercept is close to zero (5.51e-05), which is expected when using NQT transformed variables which are centered around zero. It is not statistically significant (p = 1.000).
    *   **PGR QPath\_nqt:** The coefficient is -0.0387. It is **not statistically significant** (p = 0.796). This suggests that, when controlling for the other variables, there is no significant linear relationship between NQT transformed PGR QPath and NQT transformed FOXO1 Qpath.
    *   **Serum E2 (pg/ml)\_nqt:** The coefficient is -0.6543. It is **statistically significant** (p = 0.001). This indicates that, when controlling for the other variables, there is a significant negative linear relationship between NQT transformed Serum E2 and NQT transformed FOXO1 Qpath. For a one-unit increase in NQT transformed Serum E2, NQT transformed FOXO1 Qpath is estimated to decrease by 0.6543 units.
    *   **Serum P4 (ng/ml)\_nqt:** The coefficient is 0.2413. It is **not statistically significant** (p = 0.192). This suggests that, when controlling for the other variables, there is no significant linear relationship between NQT transformed Serum P4 and NQT transformed FOXO1 Qpath.

*   **Assumption Checks (based on previous steps):**
    *   **Linearity:** Visual inspection of scatter plots (in cell `e8579f3f`) suggested approximate linearity after NQT transformation.
    *   **Homoscedasticity:** The Breusch-Pagan test (p-value = 0.1758 in cell `fe981ad6`) suggests that the assumption of homoscedasticity is met (p >= 0.05).
    *   **Normality of Residuals:** The Shapiro-Wilk test (p-value = 0.6634 in cell `Z17RDB47Jx-U`) indicates that the assumption of normality of residuals is met (p >= 0.05).
    *   **Multicollinearity:** VIF values (all below 2 in cell `JB52QeIiKvxj`) indicate no significant multicollinearity.

*   **Summary for Model 1:** The multiple linear regression model with NQT transformed variables is statistically significant and explains about 29.2% of the variance in NQT transformed FOXO1 Qpath. Among the predictors, only NQT transformed Serum E2 is a significant negative predictor. The key regression assumptions (linearity, homoscedasticity, normality of residuals, multicollinearity) appear to be reasonably met after NQT transformation.

### Model 2: Dependent Variable = PGR QPath\_nqt

(Based on the output of cell `RbOwgjWmODKX`)

*   **Overall Model Fit:**
    *   **R-squared = 0.183:** This means that approximately 18.3% of the variance in the Normal Quantile Transformed PGR QPath can be explained by the NQT transformed independent variables (Serum E2 (pg/ml)\_nqt, Serum P4 (ng/ml)\_nqt, and FOXO1 Qpath\_nqt).
    *   **Adjusted R-squared = 0.120:** Adjusted R-squared value.
    *   **F-statistic = 2.904, Prob (F-statistic) = 0.0468:** The overall model is statistically significant (p < 0.05). This indicates that at least one of the independent variables significantly predicts the NQT transformed PGR QPath.

*   **Individual Predictors:**
    *   **const (Intercept):** Close to zero (8.835e-05) and not statistically significant (p = 1.000).
    *   **Serum E2 (pg/ml)\_nqt:** The coefficient is 0.3623. It is **not statistically significant** (p = 0.117).
    *   **Serum P4 (ng/ml)\_nqt:** The coefficient is 0.0587. It is **not statistically significant** (p = 0.770).
    *   **FOXO1 Qpath\_nqt:** The coefficient is -0.0447. It is **not statistically significant** (p = 0.796).

*   **Assumption Checks (based on previous steps and this model's characteristics):**
    *   **Linearity:** Visual inspection of scatter plots of NQT variables (similar to those in cell `e8579f3f` but with PGR as Y) would be needed.
    *   **Homoscedasticity:** A Breusch-Pagan test for this specific model would be needed.
    *   **Normality of Residuals:** A Shapiro-Wilk test for this specific model's residuals would be needed.
    *   **Multicollinearity:** VIF values (all below 2.5 in cell `7sx_woa8RK6e`) indicate no significant multicollinearity among Serum E2\_nqt, Serum P4\_nqt, and FOXO1 Qpath\_nqt.

*   **Summary for Model 2:** The multiple linear regression model with NQT transformed variables is statistically significant and explains about 18.3% of the variance in NQT transformed PGR QPath. None of the individual predictors (NQT transformed Serum E2, Serum P4, or FOXO1 Qpath) are statistically significant predictors in this model at the 0.05 level. Multicollinearity is not a significant issue. Further checks for linearity, homoscedasticity, and normality of residuals specifically for this model would be beneficial, although the NQT transformation of the dependent variable is expected to improve residual normality.

## Statistical Methodology

All statistical analyses were performed using Python with the pandas, numpy, matplotlib, statsmodels, scipy, and pingouin libraries. A significance level of $\alpha = 0.05$ was used for all tests.

Descriptive statistics, including mean, standard deviation (SD), median, range, and sample size (n), were calculated for all quantitative variables within each study group (Control, IVF, HRT).

The normality of the distribution for each quantitative variable was assessed using D'Agostino's K-squared test. Based on the normality results and assessment of homogeneity of variances (using Levene's test where appropriate), between-group comparisons were performed using one-way Analysis of Variance (ANOVA) for normally distributed data with equal variances, Welch's ANOVA for normally distributed data with unequal variances, and the non-parametric Kruskal-Wallis H-test for non-normally distributed data.

The association between categorical variables (Group and HE Dating) was examined using the Chi-squared test of independence. Fisher's exact test was considered if expected cell counts were less than five.

To address violations of the normality assumption for certain variables, data transformations were explored, including Log transformation, Box-Cox transformation, and Normal Quantile Transformation (NQT). The effectiveness of these transformations was evaluated using D'Agostino's K-squared test on the transformed data.

Multiple linear regression analysis was performed to investigate the relationships between selected variables. Two primary models were examined:
1.  With FOXO1 Qpath as the dependent variable and Serum E2 (pg/ml), Serum P4 (ng/ml), and PGR QPath as independent variables.
2.  With PGR QPath as the dependent variable and Serum E2 (pg/ml), Serum P4 (ng/ml), and FOXO1 Qpath as independent variables.

Prior to regression analysis, key assumptions were assessed:
*   Linearity was visually inspected using scatter plots between the dependent and independent variables.
*   Homoscedasticity (constancy of residual variance) was assessed visually with scatter plots of residuals versus predicted values and formally using the Breusch-Pagan test.
*   The normality of residuals was evaluated using histograms, Q-Q plots, and the Shapiro-Wilk test.
*   Multicollinearity among independent variables was assessed using the Variance Inflation Factor (VIF).

Data transformations were applied to the dependent variable when the normality of residuals assumption was violated, and regression analysis was re-run with the transformed variable.

Regression model results were interpreted based on the estimated coefficients, standard errors, p-values, and the overall model fit (R-squared and F-statistic).